In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
import numpy as np
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
n_sig_lowpt = 10
n_sig_highpt = 10
pt_min, pt_max = 0, 6

In [3]:
N_files = 10
run_number = 19990

In [4]:
N_centr = 5
N_sec = 8
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path=f"/media/yoren/T7_Shield/PHENIX/Dielectrons/fulls/emcal_{run_number}/"
file_names=["my-411601.root"] 
sim_file_path="../sim/output/Newembed/sngl/"
out_file_path=f"/media/yoren/T7_Shield/PHENIX/Dielectrons/fulls/emcal_{run_number}_out/" 
legend_names = ["real","#pi^{0}","#eta","#gamma #rightarrow ee","c#bar{c}","b#bar{b}"] 
hist_select_3D_names = [f"emcal_hist_{i}" for i in range(N_sec)]

In [5]:
N_hists = len(hist_select_3D_names)
loc = True
Save_to_html = False

In [6]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


infile.Close()

/media/yoren/T7_Shield/PHENIX/Dielectrons/fulls/emcal_19990/my-411601.root


In [7]:
pt_min, pt_max = 3, 6
iFile = 0
type_loc = 1
c0 = root.TCanvas(f"c0",f"c0",1550,2*900)
c0.Divide(3,3)
projz = hists_read[iFile][0].ProjectionZ(f"projz{type_loc}")
legends = []
emcal_pixel_maps = []
for isector in range(len(hists_read[iFile])):
    c0.cd(1+isector)
    emcal_pixel_maps.append(root.TH2D(f"new_hist_{isector}",f"new_hist_{isector};z;y",hists_read[iFile][isector].GetNbinsX(), hists_read[iFile][isector].GetXaxis().GetBinLowEdge(1),\
                                hists_read[iFile][isector].GetXaxis().GetBinUpEdge(hists_read[iFile][isector].GetNbinsX()),hists_read[iFile][isector].GetNbinsY(),\
                                hists_read[iFile][isector].GetYaxis().GetBinLowEdge(1),hists_read[iFile][isector].GetYaxis().GetBinUpEdge(hists_read[iFile][isector].GetNbinsY())))
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    for ibinz in range(ibinz0,1+ibinz1):
        for ibinx in range(1,1+hists_read[iFile][isector].GetNbinsX()):
            for ibiny in range(1,1+hists_read[iFile][isector].GetNbinsY()):
                emcal_pixel_maps[-1].SetBinContent(ibinx,ibiny,emcal_pixel_maps[-1].GetBinContent(ibinx,ibiny)+hists_read[iFile][isector].GetBinContent(ibinx,ibiny,ibinz))


    emcal_pixel_maps[-1].Draw("colz")

c0.Draw()

In [8]:
def find_hot_pixels(h2,
                    half_win=3,        # neighborhood half-size (3 -> 7x7 window)
                    nsig=n_sig_lowpt,          # threshold in robust sigmas
                    min_neighbors=10,  # minimum valid neighbors to trust stats
                    min_excess=n_sig_lowpt,    # absolute excess above local median
                    zero_cut=0.0,      # neighbors <= zero_cut are treated as dead/cold
                    make_zmap=True):   # build a TH2D with robust z-scores
    nx, ny = h2.GetNbinsX(), h2.GetNbinsY()
    xax, yax = h2.GetXaxis(), h2.GetYaxis()

    zmap = None
    if make_zmap:
        zmap = h2.Clone(h2.GetName()+"_zmap")
        zmap.Reset()
        zmap.SetTitle(h2.GetTitle()+";"+xax.GetTitle()+";"+yax.GetTitle()+";robust z")

    hot_xy = []
    for ix in range(1, nx+1):
        for iy in range(1, ny+1):
            c = h2.GetBinContent(ix, iy)

            # build neighborhood (exclude center and dead/cold neighbors)
            nb = []
            for dx in range(-half_win, half_win+1):
                jx = ix + dx
                if jx < 1 or jx > nx: 
                    continue
                for dy in range(-half_win, half_win+1):
                    jy = iy + dy
                    if jy < 1 or jy > ny or (dx == 0 and dy == 0):
                        continue
                    v = h2.GetBinContent(jx, jy)
                    if v > zero_cut:
                        nb.append(v)

            if len(nb) < min_neighbors:
                if zmap: zmap.SetBinContent(ix, iy, 0.0)
                continue

            nb = np.asarray(nb, dtype=float)
            med = np.median(nb)
            mad = np.median(np.abs(nb - med))
            sigma = 1.4826*mad  # MAD -> sigma (Gaussian)
            # floor sigma to something sensible (helps tiny-stat regions)
            if sigma < 1.0:
                sigma = max(1.0, np.sqrt(max(med, 1.0)))

            z = (c - med)/sigma if sigma > 0 else 0.0
            if zmap: zmap.SetBinContent(ix, iy, z)

            if (c - med) >= min_excess and z >= nsig:
                x = xax.GetBinCenter(ix)
                y = yax.GetBinCenter(iy)
                hot_xy.append((x, y, c, med, z))

    # overlay as a TGraph
    if len(hot_xy) == 0:
        return None, zmap, hot_xy
    g = root.TGraph(len(hot_xy))
    for i, (x, y, c, med, z) in enumerate(hot_xy):
        g.SetPoint(i, x, y)
    g.SetName(h2.GetName()+"_hotpixels")
    g.SetTitle("Hot pixels")
    g.SetMarkerStyle(31)   # star
    g.SetMarkerSize(1.6)
    g.SetMarkerColor(root.kRed+1)

    return g, zmap, hot_xy


In [9]:
iFile = 0
type_loc = 1
pt_min, pt_max = 2, 6
c0 = root.TCanvas("c0","c0",1550,2*900)
c0.Divide(3,3)

projz = hists_read[iFile][0].ProjectionZ(f"projz{type_loc}")
emcal_pixel_maps = []
hot_overlays = []
z_maps = []

for isector in range(len(hists_read[iFile])):
    c0.cd(1+isector)

    src = hists_read[iFile][isector]
    h2 = root.TH2D(f"new_hist_{isector}", f"new_hist_{isector};{src.GetXaxis().GetTitle()};{src.GetYaxis().GetTitle()}",
                   src.GetNbinsX(), src.GetXaxis().GetBinLowEdge(1), src.GetXaxis().GetBinUpEdge(src.GetNbinsX()),
                   src.GetNbinsY(), src.GetYaxis().GetBinLowEdge(1), src.GetYaxis().GetBinUpEdge(src.GetNbinsY()))
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    for ibinz in range(ibinz0, ibinz1+1):
        for ibinx in range(1, src.GetNbinsX()+1):
            for ibiny in range(1, src.GetNbinsY()+1):
                h2.SetBinContent(ibinx,ibiny,h2.GetBinContent(ibinx,ibiny)+src.GetBinContent(ibinx,ibiny,ibinz))

    emcal_pixel_maps.append(h2)

    # --- detect hot pixels (tune params if needed) ---
    g, zmap, hot_list = find_hot_pixels(h2,
                                        half_win=3,    # 7x7 local window
                                        nsig=10.0,      # ~5σ above local bg
                                        min_neighbors=10,
                                        min_excess=10.0,
                                        zero_cut=0.0,
                                        make_zmap=True)
    hot_overlays.append(g)
    z_maps.append(zmap)

    minim = emcal_pixel_maps[-1].Integral() / ( 71*36 if isector<6 else 96*48 ) / 5
    emcal_pixel_maps[-1].GetZaxis().SetRangeUser(minim, emcal_pixel_maps[-1].GetMaximum()*1.1)
    root.gPad.SetLogz()
    # draw base map + overlay markers
    emcal_pixel_maps[-1].Draw("COLZ")
    if g and g.GetN() > 0:
        g.Draw("P SAME")

c0.Draw()

# If you want to list them:
for isector, hot_list in enumerate([hl for (_,_,hl) in [find_hot_pixels(h2, make_zmap=False) for h2 in emcal_pixel_maps]]):
    if not hot_list: 
        continue
    print(f"Sector {isector}: {len(hot_list)} hot pixels")
    for (x,y,c,med,z) in hot_list:
        print(f"  x={x:.2f}, y={y:.2f}, content={c:.1f}, local_med={med:.1f}, z={z:.2f}")


In [10]:
# --- assumes you already filled emcal_pixel_maps[sector] as in your code ---
# and you have the find_hot_pixels() from earlier.
# If you don't, paste that function first.

def _hot_bin_indices(h2, **kwargs):
    """Return list of (ix0, iy0) zero-based hot bin indices using the robust detector."""
    # we call the robust detector but we only need indices; avoid building the zmap
    nx, ny = h2.GetNbinsX(), h2.GetNbinsY()
    # local copy of the detector core (small inline to avoid dependency)
    half_win = kwargs.get("half_win", 3)
    nsig = kwargs.get("nsig", n_sig_lowpt)
    min_neighbors = kwargs.get("min_neighbors", 10)
    min_excess = kwargs.get("min_excess", n_sig_lowpt)
    zero_cut = kwargs.get("zero_cut", 0.0)

    hot_idx = []
    for ix in range(1, nx+1):
        for iy in range(1, ny+1):
            c = h2.GetBinContent(ix, iy)
            nb = []
            for dx in range(-half_win, half_win+1):
                jx = ix + dx
                if jx < 1 or jx > nx: continue
                for dy in range(-half_win, half_win+1):
                    jy = iy + dy
                    if jy < 1 or jy > ny: continue
                    if dx==0 and dy==0: continue
                    v = h2.GetBinContent(jx, jy)
                    if v > zero_cut: nb.append(v)
            if len(nb) < min_neighbors: continue
            nb = np.asarray(nb, float)
            med = float(np.median(nb))
            mad = float(np.median(np.abs(nb - med)))
            sigma = 1.4826*mad
            if sigma < 1.0:
                sigma = max(1.0, np.sqrt(max(med, 1.0)))
            z = (c - med)/sigma if sigma>0 else 0.0
            if (c - med) >= min_excess and z >= nsig:
                hot_idx.append((ix-1, iy-1))  # zero-based
    return hot_idx

def write_hot_txt(emcal_pixel_maps, txt_path="hot_pixels.txt",
                  sectors=None, **detector_kwargs):
    """
    Write lines: sector  iy  iz  isdead
    Note: iz corresponds to X (z-axis) bin index; iy corresponds to Y bin index.
    """
    if sectors is None:
        sectors = range(len(emcal_pixel_maps))
    rows = []
    for sec in sectors:
        h2 = emcal_pixel_maps[sec]
        hot_idx = _hot_bin_indices(h2, **detector_kwargs)
        for (ix0, iy0) in hot_idx:
            iz0 = ix0
            rows.append((sec, iy0, iz0, 1))  # isdead==1 for flagged pixel
    rows.sort()
    with open(txt_path, "w") as f:
        for sec, iy0, iz0, flag in rows:
            f.write(f"{sec} {iy0} {iz0} {flag}\n")
    return rows  # useful for the header writer



reading file list

In [11]:
import os
import re

file_pattern = re.compile(r"my-(\d{6})\.root")

file_numbers = []

for filename in os.listdir(file_path):
    match = file_pattern.match(filename)
    if match:
        file_numbers.append(match.group(1))

print(len(file_numbers))

1065


In [12]:
file_numbers.sort()

In [13]:
#file_numbers= file_numbers[:10]
#print(file_numbers)

In [14]:
root.gStyle.SetOptStat(0)

In [ ]:
def vertex_estimator(file_number, N_hists_local = [0]):
    infile = root.TFile.Open(file_path+f"my-{file_number}.root", "read")
    hists_read_file = []
    for ihist in N_hists_local:
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{file_number}")
    hitst_stat = infile.Get("event_hist")
    hitst_stat.SetDirectory(root.nullptr)
    c0 = root.TCanvas("c0","c0",2100,1100)
    c0.Divide(4,2)

    projz = hists_read_file[0].ProjectionZ(f"projz{file_number}")
    emcal_pixel_maps = []
    hot_overlays = []
    z_maps = []
    pt_min, pt_max = 2, 6

    for isector in range(len(hists_read_file)):
        c0.cd(1+isector)

        src = hists_read_file[isector]
        h2 = root.TH2D(f"new_hist_{isector}", f"new_hist_{isector};{src.GetXaxis().GetTitle()};{src.GetYaxis().GetTitle()}",
                       src.GetNbinsX(), src.GetXaxis().GetBinLowEdge(1), src.GetXaxis().GetBinUpEdge(src.GetNbinsX()),
                       src.GetNbinsY(), src.GetYaxis().GetBinLowEdge(1), src.GetYaxis().GetBinUpEdge(src.GetNbinsY()))
        ibinz0 = projz.FindBin(pt_min)
        ibinz1 = projz.FindBin(pt_max)-1
        for ibinz in range(ibinz0, ibinz1+1):
            for ibinx in range(1, src.GetNbinsX()+1):
                for ibiny in range(1, src.GetNbinsY()+1):
                    h2.SetBinContent(ibinx,ibiny,h2.GetBinContent(ibinx,ibiny)+src.GetBinContent(ibinx,ibiny,ibinz))

        emcal_pixel_maps.append(h2)

        # --- detect hot pixels (tune params if needed) ---
        g, zmap, hot_list = find_hot_pixels(h2,
                                            half_win=3,    # 7x7 local window
                                            nsig=10.0,      # ~5σ above local bg
                                            min_neighbors=10,
                                            min_excess=10.0,
                                            zero_cut=0.0,
                                            make_zmap=True)
        hot_overlays.append(g)
        z_maps.append(zmap)

        minim = 1.
        minim = emcal_pixel_maps[-1].Integral() / ( 71*36 if isector<6 else 96*48 ) / 5
        emcal_pixel_maps[-1].GetZaxis().SetRangeUser(minim, emcal_pixel_maps[-1].GetMaximum()*1.1)
        root.gPad.SetLogz()

        # draw base map + overlay markers
        emcal_pixel_maps[-1].Draw("COLZ")
        if g and g.GetN() > 0:
            g.Draw("P SAME")

    c0.Draw()
    c0.SaveAs(out_file_path+f"emcal_hot_pixels_{file_number}.png")


    rows = write_hot_txt(
        emcal_pixel_maps,
        txt_path=out_file_path+f"hot_pixels{file_number}.txt",
        half_win=3, nsig=n_sig_lowpt, min_neighbors=10, min_excess=n_sig_lowpt, zero_cut=0.0
    )

    return rows


In [16]:
#file_numbers = file_numbers[:10]
#print(file_numbers)
#file_numbers = ['411010', '411152', '411354', '411561', '411602', '411819', '411926', '412068', '412328', '412412', '412879', '413077', '413079', '413088', '413187', '413242', '413540', '413561', '413631', '413662', '413747', '413931', '413934', '414061', '414076', '414135', '414148', '414207', '414331', '414606', '414626', '414634', '414825', '414829', '414885', '414886']

In [ ]:
means, sigmas, mean_errs, sigma_errs, entries = [[] for _ in range(N_sec)], [[] for _ in range(N_sec)], [[] for _ in range(N_sec)], [[] for _ in range(N_sec)], [[] for _ in range(N_sec)]
for file_number in file_numbers:
    params_for_all_secotrs = vertex_estimator(file_number, N_hists_local = [0,1,2,3,4,5,6,7])